In [1]:
def prepare_data():
    detector = cv2.CascadeClassifier('/home/shubham/Desktop/face reco/face recognition advance/haarcascade_frontalface_default.xml')
    if not os.path.isdir('/home/shubham/Desktop/face reco/face recognition advance/dataset'):
        os.mkdir('/home/shubham/Desktop/face reco/face recognition advance/dataset')
    os.chdir('/home/shubham/Desktop/face reco/face recognition advance/dataset')
    id = int(input('Enter ID : '))
    name = input('Enter Name : ')
    sampleno = 0
    opt = input("Capture face through Photo or Camera(p/c) : ")
    if opt=='c':
        cam = cv2.VideoCapture(0)
        while(True):
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
            faces = detector.detectMultiScale(gray, 1.2, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
                sampleno+=1
                cv2.imwrite(str(id)+'_'+name+'.'+str(sampleno)+'.jpg', gray[y:y+h, x:x+w])
                cv2.imshow('Capturing', img)
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
            elif sampleno>=40:
                break
        cam.release()
        cv2.destroyAllWindows()
    elif opt=='p':
        path = input('Enter path with name : ')
        img = cv2.imread(path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        faces = detector.detectMultiScale(gray, 1.2, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 4)
            cv2.imwrite(str(id)+'_'+name+'.jpg', gray[y:y+h, x:x+w])
            cv2.imshow('Capturing', img)
        cv2.waitKey(100)
        cv2.destroyAllWindows()

In [2]:
def training_data():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    path = '/home/shubham/Desktop/face reco/face recognition advance/dataset'
    images = [os.path.join(path,file) for file in os.listdir(path)]
    faces = []
    IDs = []
    Names = []
    for i in images:
        img = Image.open(i).convert('L')
        face_info = np.array(img, dtype='uint8')
        temp_info = os.path.split(i)[-1].split('_')
        faces.append(face_info)
        IDs.append(int(temp_info[0]))
        Names.append(str(temp_info[1]).split('.')[0])
    recognizer.train(faces,np.array(IDs))
    if not os.path.isdir('/home/shubham/Desktop/face reco/face recognition advance/trained data'):
        os.mkdir('/home/shubham/Desktop/face reco/face recognition advance/trained data')
    recognizer.save('/home/shubham/Desktop/face reco/face recognition advance/trained data/trained_data.yml')
    cv2.destroyAllWindows()
    return Names,IDs

In [3]:
def recognize(Names, IDs):
    detector = cv2.CascadeClassifier('/home/shubham/Desktop/face reco/face recognition advance/haarcascade_frontalface_default.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read('/home/shubham/Desktop/face reco/face recognition advance/trained data/trained_data.yml')
    opt = input('Detect Through Photo or Camera(p/c): ')
    if opt=='p':
        path = input('Enter path with name : ')
        img = cv2.imread(path)
        if img.shape[0]>1000 or img.shape[1]>1000:
            img = cv2.resize(img, (int((img.shape[0]/2)/1.5), int(img.shape[0]/2)), interpolation=cv2.INTER_AREA)
        elif img.shape[0]>2000 or img.shape[1]>2000:
            img = cv2.resize(img, (int((img.shape[0]/4)/1.5), int(img.shape[0]/4)), interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
        faces = detector.detectMultiScale(gray, 1.2, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
            id, conf = recognizer.predict(gray[y:y+h, x:x+w])
            put_name = str(Names[IDs.index(id)])
            cv2.putText(img, str(id)+' '+put_name, (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 2)
        cv2.imshow('Recognized', img)
        f = cv2.waitKey(0)
        if f==ord('q'):
            cv2.destoryAllWindows()
    elif opt=='c':
        cam = cv2.VideoCapture(0)
        while(True):
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)
            faces = detector.detectMultiScale(gray, 1.2, 5)
            for (x,y,w,h) in faces:
                cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
                id, conf = recognizer.predict(gray[y:y+h, x:x+w])
                put_name = str(Names[IDs.index(id)])
                cv2.putText(img, str(id)+' '+put_name, (x,y+h+40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Recognized', img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cam.release()
        cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
from IPython.display import clear_output
if not os.path.isdir('/home/shubham/Desktop/face reco/face recognition advance'):
    os.mkdir('/home/shubham/Desktop/face reco/face recognition advance')
    os.mkdir('/home/shubham/Desktop/face reco/face recognition advance/dataset')
    os.mkdir('/home/shubham/Desktop/face reco/face recognition advance/trained data')
if os.listdir('/home/shubham/Desktop/face reco/face recognition advance/dataset'):
    Names, IDs = training_data()
while(True):
    print('\t\tFACE RECOGNITION\t\tTo quit press Q\n')
    print('1) Capture Face')
    print('2) Train Data')
    print('3) Recognize Face')
    opt = input('Choose an option : ')
    if opt=='1':
        prepare_data()
    elif opt=='2':
        Names, IDs = training_data()
    elif opt=='3':
        recognize(Names, IDs)
    elif opt=='q':
        break
    else:
        print("Invalid Option")
    clear_output()
        

		FACE RECOGNITION		To quit press Q

1) Capture Face
2) Train Data
3) Recognize Face
Choose an option : 1
Enter ID : 121
Enter Name : shubham
Capture face through Photo or Camera(p/c) : c
